<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/chatbot_with_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Korpora

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/ChatbotData.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [ ]:
# 질문과 답변 문장을 따로 저장

texts = []
pairs = []
for text, pair in zip(df['Q'], df['A']) :
  texts.append(text)
  pairs.append(pair)


In [ ]:
# 질문과 답변 쌍을 5개 확인
list(zip(texts, pairs))[:5]

[('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.')]

In [ ]:
## 빠르고 간단한 테스트를 위해 특수문자와 영어 제거

import re

def clean_sentence(sentence) :
  sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', r'', sentence)
  return sentence

In [ ]:
## 위 함수 잘 작동하는지 테스트
sentence = clean_sentence('12시 땡^^!!??')


In [ ]:
## 한국어 문장을 분해하기 위한 라이브러리(형태소 분석)
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.8 MB/s eta 0:00:00


In [ ]:
## 형태소 추출
### 형태소란? 의미를 가지는 요소로서는 더 이상 쪼갤 수 없는 가장 작은 말의 단위
from konlpy.tag import Okt

okt = Okt()

def process_morph(sentence):
  return ' '.join(okt.morphs(sentence))


In [ ]:
## 위 함수 잘 작동하는지 확인
process_morph('안녕하세요 저는 홍길동입니다. 당신의 성공을 항상 기원합니다. 사랑합니다.')

'안녕하세요 저 는 홍길동 입니다 . 당신 의 성공 을 항상 기원 합니다 . 사랑 합니다 .'

In [ ]:
## 문장을 입력받아 형태소로 쪼개주는 함수
def clean_and_morph(sentence, is_question=True):
  ## 한글만 남기기
  sentence = clean_sentence(sentence)

  ## 형태소로 쪼개기
  sentence = process_morph(sentence)

  if is_question:
    return sentence

  else :
    ## 후에 토크나이저하기 위해서는 공백이 꼭 들어가야 함.
    return ('<START> ' + sentence, sentence + ' <END>')

In [ ]:
def preprocess(texts, pairs):
  questions = []
  answer_in = []
  answer_out = []

  ## 질문에 대한 전처리
  for text in texts :
    question = clean_and_morph(text, is_question=True)
    questions.append(question)

  ## 답변에 대한 전처리
  for pair in pairs:
    in_, out_ = clean_and_morph(pair, is_question=False)
    answer_in.append(in_)
    answer_out.append(out_)

  return questions, answer_in, answer_out


In [ ]:
q, ai, ao = preprocess(texts[:3], pairs[:3])


In [ ]:
q
ai
ao

len(questions)

In [ ]:
texts = texts[:1000]
pairs = pairs[:1000]
questions, answer_in, answer_out = preprocess(texts, pairs)

In [ ]:
questions[:5]

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', '심하네']

In [ ]:
answer_in[:5]

['<START> 하루 가 또 가네요',
 '<START> 위로 해 드립니다',
 '<START> 여행 은 언제나 좋죠',
 '<START> 여행 은 언제나 좋죠',
 '<START> 눈살 이 찌푸려지죠']

In [ ]:
answer_out[:5]

['하루 가 또 가네요 <END>',
 '위로 해 드립니다 <END>',
 '여행 은 언제나 좋죠 <END>',
 '여행 은 언제나 좋죠 <END>',
 '눈살 이 찌푸려지죠 <END>']

In [ ]:
## 후에 토크나이저를 한번에 하기 위해 문장을 합쳐줌(리스트 합치기)
all_sentences = questions + answer_in + answer_out

In [ ]:
all_sentences[-1]

'신나는 노래 로 분위기 를 띄어 보세요 <END>'

In [ ]:
# 전체 형태소 개수
a = (' '.join(questions) + ' '.join(answer_in) + ' '.join(answer_out)).split()
len(set(a))

2300

In [ ]:
import warnings
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
## filter => 문장의 특수기호등을 임의로 처리하지 말라. 필터링 하지 말라
## lower => 소문자로 변경하지 마라
## oov_token => 단어 사전에 존재하지 않는 단어라면 '<OOV>'로 대체
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')

In [ ]:
## 단어 사전 만들기
## 공백을 기준으로 쪼개주는 듯하다
tokenizer.fit_on_texts(all_sentences)

In [ ]:
## 각 단어와 단어의 인덱스 번호를 확인
for word, idx in tokenizer.word_index.items():
  print(f'{word}\t\t => \t{idx}')
  if idx > 10:
    break

<OOV>		 => 	1
<START>		 => 	2
<END>		 => 	3
이		 => 	4
거		 => 	5
을		 => 	6
가		 => 	7
나		 => 	8
예요		 => 	9
사람		 => 	10
요		 => 	11


In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
VOCAB_SIZE

2301

In [ ]:
## 문자형태를 숫자 형태로 바꾸기
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [ ]:
## 문자가 숫자로 바뀐 것 확인
questions[0], answer_in_sequence[0],answer_out_sequence[0], pairs[0], tokenizer.word_index['하루']

('12시 땡', [2, 391, 7, 356, 1234], [391, 7, 356, 1234, 3], '하루가 또 가네요.', 391)

In [ ]:
## 딥러닝의 경우 입력값이 항상 일정해야 하므로(네트워크 모델은 입력값에 의해 모양이 바뀌므로 입력값은 바뀌면 안된다.)
MAX_LENGTH = 30 # 최대 몇개의 단어
TRUNCATING = 'post' # 잘라낼 때 앞(pre), 뒤(post)
PADDING = 'post' # 채워줄 때 앞(pre), 뒤(post)

In [ ]:
## 트런케이팅과 패딩 적용하기
question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)
answer_in_padded = pad_sequences(answer_in_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)
answer_out_padded = pad_sequences(answer_out_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)

In [ ]:
question_padded.shape, answer_in_padded.shape, answer_out_padded.shape

((1000, 30), (1000, 30), (1000, 30))

In [ ]:
question_padded[0]

#tokenizer.word_index['12시']
#tokenizer.word_index['땡']


array([1608, 1609,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
## 단어에는 비교우위가 없으므로 카테고리컬로 데이터로 보고 원핫 인코딩을 해준다.
def convert_to_one_hot(padded) :
  one_hot_vector = np.zeros((len(padded), MAX_LENGTH, VOCAB_SIZE))

  for i, sequence in enumerate(padded):
    for j, index in enumerate(sequence):
      one_hot_vector[i, j, index] = 1

  return one_hot_vector

answer_in_padded[:5]

array([[   2,  391,    7,  356, 1234,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   2,  143,   36,  414,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   2,  106,   19,  206,   86,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   2,  106,   19,  206,   86,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   2, 1235,    4, 1236,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 

In [ ]:
tokenizer.word_index.values()

In [ ]:
from tensorflow.keras.utils import to_categorical

# 우선, 단어 인덱스의 최대값을 찾아야 합니다.
# 이 값이 원-핫 벡터의 길이가 됩니다.
max_word_idx = 12637  # `data`는 단어 인덱스를 담고 있는 (11823, 30) 형태의 numpy 배열입니다.

# 원-핫 인코딩 수행
data_one_hot = to_categorical(answer_in_padded, num_classes=max_word_idx + 1)


In [ ]:
#np.equal(data_one_hot, answer_in_one_hot)
#data_one_hot.shape, answer_in_one_hot.shape

In [ ]:
answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)

In [ ]:
answer_in_one_hot.shape

(1000, 30, 2301)

In [ ]:
answer_in_one_hot[0].shape, answer_out_one_hot[0].shape

((30, 2301), (30, 2301))

In [ ]:
## 모델이 예측한 인코딩된 값을 다시 문자로 디코딩 해주는 함수
def convert_index_to_text(indexes, end_token):
  sentence = ''
  for index in indexes: ## 문장의 순서
    if index == end_token:  ## 문장의 마지막이면 종료
      break
    if index > 0 and tokenizer.index_word[index] is not None: ## 단아 사전에 존재하고 올바른 인덱스라면
      sentence += tokenizer.index_word[index] # 최종 문자열에 이어 붙인다.
    else:
      sentence += '' # 없는 거면 공백.

    sentence += ' ' # 한 형태소가 끝나면 띄어쓰기
  return sentence

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
## 모델 객체
class Encoder(tf.keras.Model):
  def __init__(self, units, vocab_size, embedding_dim, time_steps):
    super(Encoder, self).__init__()

    ## Embedding -> 카테고리컬 단어값을 고차원으로 바꾸는 것(우리는 원핫을 사용함)
    self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps) ## 단어 개수, 변환하고자 하는 임베딩 차원, 한 문장의 길이
    self.dropout = Dropout(0.2) ## 과적합을 방지하기 위한 하이퍼파라미터. 임의로 20% 뉴런을 잡아서 비활성화 시킴
    self.lstm = LSTM(units, return_state=True) ## 최종 히든 스테이트를 얻어야 벡터 콘텍스트에 넣을 수 있음.

  def call(self, inputs):
    x = self.embedding(inputs) ## 임베딩 세팅
    x = self.dropout(x) ## 과적합 방지 파라미터 세팅
    x, hidden_state, cell_state =self.lstm(x) ## 정답, 히든 스테이트, 셀 스테이트

    return [hidden_state, cell_state]




In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, units, vocab_size, embedding_dim, time_steps):
    super(Decoder, self).__init__()
    self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
    self.dropout = Dropout(0.2)
    self.lstm = LSTM(units,
                     return_state=True, # 스테이트값을 알아야 다음 셀에서 진행 가능
                     return_sequences=True ## 각 유닛의 스테이트값을 다 얻어서 결과를 얻어야 하므로
    )
    self.dense = Dense(vocab_size, activation='softmax') ## 결과를 얻기 위한 출력층

  def call(self, inputs, initial_state): ## initial_state는 encoder의 출력값
    x = self.embedding(inputs)
    x = self.dropout(x)
    x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
    x = self.dense(x) # 최종 결과값은 출력층을 거쳐 결과를 낸다

    return x, hidden_state, cell_state



In [ ]:
class Seq2seq(tf.keras.Model):
  def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
    super(Seq2seq, self).__init__()

    self.start_token = start_token
    self.end_token = end_token
    self.time_steps = time_steps # 문장의 길이(30)

    self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
    self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

  def call(self, inputs, training=True): ## training: 학습용, 예측용 구별
    if training: ## 학습일 땐,
      encoder_inputs, decoder_inputs = inputs ## 인코더, 디코더 모두 동일한 입력값 넣는다.
      context_vector = self.encoder(encoder_inputs) ## 인코더에 넣어서 벡터 얻어냄
      decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, initial_state=context_vector) ## 얻어낸 인코더의 벡터값을 디코더에 사용

      return decoder_outputs

    else: ## 예측일 땐,
      context_vector = self.encoder(inputs) ##
      target_seq = tf.constant([[self.start_token]], dtype=tf.float32) ## 첫번째는 무조건 <START>,
      results = tf.TensorArray(tf.int32, self.time_steps) ## 결과 배열. 그래프 그리기 위해 텐서 배열로 담는다.

      for i in tf.range(self.time_steps):
        decoder_output, decoder_hidden, decoder_cell = self.decoder(target_seq, initial_state=context_vector)
        decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
        decoder_output = tf.reshape(decoder_output, shape=(1, 1))
        results = results.write(i, decoder_output)

        if decoder_output == self.end_token:
          break

        target_seq = decoder_output
        context_vector = [decoder_hidden, decoder_cell]

      return tf.reshape(results.stack(), shape=(1, self.time_steps))

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH

START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']
UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index) + 1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
NUM_EPOCHS = 10


In [ ]:
tokenizer.word_index

In [ ]:
checkpoint_path = 'sample-checkpoint.h5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1,
                             save_weights_only=True)

In [ ]:
seq2seq = Seq2seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
def make_prediction(model, question_inputs) :
  results = model(inputs=question_inputs, training=False)
  results = np.asarray(results).reshape(-1)
  return results



In [ ]:
for epoch in range(NUM_EPOCHS):
  print(f"processing epoch : {epoch * 10 + 1} ...")
  seq2seq.fit([question_padded, answer_in_padded], answer_out_one_hot, epochs=10, batch_size=BATCH_SIZE, callbacks=[checkpoint])

  samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

  for idx in samples:
    question_inputs = question_padded[idx]
    results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))

    results = convert_index_to_text(results, END_TOKEN)

processing epoch : 1 ...
Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 3.2577 - acc: 0.7767
Epoch 1: loss improved from inf to 3.25767, saving model to sample-checkpoint.h5
63/63 [==============================] - 12s 87ms/step - loss: 3.2577 - acc: 0.7767
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 1.3077 - acc: 0.8013
Epoch 2: loss improved from 3.25767 to 1.30768, saving model to sample-checkpoint.h5
63/63 [==============================] - 6s 93ms/step - loss: 1.3077 - acc: 0.8013
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 1.2311 - acc: 0.8068
Epoch 3: loss improved from 1.30768 to 1.23105, saving model to sample-checkpoint.h5
63/63 [==============================] - 6s 91ms/step - loss: 1.2311 - acc: 0.8068
Epoch 4/10
63/63 [==============================] - ETA: 0s - loss: 1.1485 - acc: 0.8196
Epoch 4: loss improved from 1.23105 to 1.14854, saving model to sample-checkpoint.h5
63/63 [==========================

In [ ]:
seq2seq

In [ ]:
seq2seq.save_weights('test')

In [ ]:
def make_question(sentence):
  sentence = clean_and_morph(sentence)
  question_sequence = tokenizer.texts_to_sequences([sentence])
  question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)

  return question_padded

In [ ]:
def run_chatbot(question):
  question_inputs = make_question(question)
  results = make_prediction(seq2seq, question_inputs)
  results = convert_index_to_text(results, '<END>')
  return results

In [ ]:
while True:
  user_input = input('\n말을 걸어 주세요.')
  if user_input == 'q':
    break
  answer = run_chatbot(user_input)
  print(f'챗봇 응답:{answer}\n')

In [ ]:
seq2seq3 = Seq2seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)

In [ ]:
#seq2seq.save_weights('aaa')
seq2seq3.load_weights('aaa')



In [ ]:
def run_chatbot(question):
  question_inputs = make_question(question)
  results = make_prediction(seq2seq3, question_inputs)
  results = convert_index_to_text(results, '<END>')
  return results

In [ ]:
def make_question(sentence):
  sentence = clean_and_morph(sentence)
  question_sequence = tokenizer.texts_to_sequences([sentence])
  question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)

  return question_padded

In [ ]:
run_chatbot("아오 치치!")

'저 도 궁금하네요 <END>                           '

In [ ]:
import joblib

joblib.dump(tokenizer, 'tokenizer')

['tokenizer']